<br>

# Introdução

- https://www.tjsp.jus.br/ListaTelefonica


In [ ]:
import html
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from more_itertools import one
from open_geodata import geo
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from traquitanas.scrapping import adds, gecko

from sp_tjsp_divadmin.my_driver import Driver
from sp_tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
)
from sp_tjsp_divadmin.my_paths import (
    adds_path,
    driver_path,
    logs_path,
    output_path_tab,
)

<br>

## Driver

Inicialmente, na primeira abordagem, tentei acessar o site da Lista Telefônica para obter os dados do TJSP.


In [ ]:
driver = Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,
    verify_ssl=False,
)
driver.add_extension_xpath(my_adds_path=adds_path)

In [ ]:
# Go TJSP
URL = 'https://www.tjsp.jus.br/ListaTelefonica'
driver.get(URL)

In [ ]:
# Parâmetros
procurar_por = 'Município'
text_procurar = 'Santos'

In [ ]:
# Tipo de Busca
select = Select(driver.find_element(By.XPATH, '//*[@id="btnTipoBusca"]'))

# Opções para Procura
option_list = select.options
option_list = [x.text for x in option_list]

# sss
if procurar_por in option_list:
    select.select_by_visible_text(procurar_por)
else:
    raise Exception(f'Valor deve estar entre {option_list}')

<br>

Dai notei que a cada letra digitada, uma lista de municípios apareciam, indicando que algum método POST estava atuando.

Optei por ver o método POST e trabalhar com ele, mudando a abordagem!


In [ ]:
field_xpath = driver.find_element(By.XPATH, '//*[@id="Descricao"]')
field_xpath.clear()
field_xpath.send_keys(Keys.BACKSPACE * 50)

for letra in text_procurar:
    print(letra)
    time.sleep(1)
    field_xpath.send_keys(letra)

In [ ]:
driver.quit()

<br>

## Lista de Municípios


In [ ]:
geo.get_dataset_names()
df_geo_mun = geo.load_dataset('tab.sp.tab_municipio_nome')
df_geo_mun.head()
lista_municipios = df_geo_mun['municipio_nome'].to_list()
len(lista_municipios)

In [ ]:
def get_lista_municipios_tjsp(municipio):
    """
    Pesquisa de municípios a partir de alguns caracteres.
    A função sempre retorna 10 itens.
    A cada caractere, o número de registros afunila!

    Exemplo de uso:
    df = get_lista_municipios_tjsp('Santos')

    :param municipio: _description_
    :type municipio: _type_
    :raises Exception: _description_
    """
    if len(municipio) < 3:
        raise Exception('A pesquisa de município deve ter mais de 3 caracteres')

    r = requests.post(
        'https://www.tjsp.jus.br/AutoComplete/ListarMunicipios',
        json={'texto': municipio},
    )
    if r.json() == 'listaVazia':
        pass
    else:
        return pd.DataFrame(r.json())

In [ ]:
# Apenas para Testes
get_lista_municipios_tjsp('Santos')
# get_lista_municipios_tjsp('Luiz A')

<br>

Obtive o município com maior número de caracteres, visto que a função do método POST sempre lista apenas 10 municípios...


In [ ]:
n_caracteres_mun_max = max([len(x) for x in lista_municipios])
n_caracteres_mun_max

<br>

Uma vez com a função que retorna os municípios escrita, fiz uma iteração por todos os trechos de nomes de municípios. Para cada municípios foi pesquisado diversas vezes, incrementando o número de caracteres. Por exemplos, o município de Santos foi pesquisado:

- San
- Sant
- Santo
- Santos

<br>

Isso garantiu que todos os municípios foram "raspados".

A FUNÇÃO LEVA 20 MINUTOS PARA RODAR!


In [12]:
list_dfs = []
for i in range(n_caracteres_mun_max)[3:]:
    lista_municipios_temp = list(
        set([mun[:i] for mun in lista_municipios if len(mun) >= i])
    )
    for search_text in lista_municipios_temp:
        print(search_text)
        df_temp = get_lista_municipios_tjsp(search_text)
        list_dfs.append(df_temp)

Alto
Auri
Tamb
Araç
Soco
Dumo
Urân
Tupi
Muru
Anal
Mogi
Iara
Turm
Tabo
Rest
Silv
Cara
Arei
Arar
Lago
Morr
Ipau
Várz
Apia
Ubar
Cunh
Cana
Uniã
Zaca
Iper
Polo
Itaq
Indi
Char
Quad
Sert
Mong
Regi
Ocau
Alti
Ubat
Mari
Boit
Jaci
Anhu
Enge
Quat
Ranc
Rosa
Paul
Pinh
Osva
Mesó
Itir
Ipor
Pira
Bast
José
Taiú
Glic
Taqu
Pená
Junq
Fart
Osca
Teod
Bana
Ipig
Artu
Tiet
Assi
Flor
Tagu
Ucho
Caco
Itaj
Cafe
Flór
Aram
Borá
Anga
Ipeú
Ribe
Valp
Taba
Pont
Crav
Sand
Arap
Rubi
Taci
Alam
Sagr
Pedr
Gene
Agua
Popu
Poti
Osas
Caiu
Torr
Nhan
Mend
Brot
Orin
Embu
Irac
Cati
Moco
Clem
Avan
Matã
Agud
Arco
Neve
Tuiu
Pita
Tupã
Ituv
Quin
Emba
Colô
Laví
Quel
Dolc
Getu
Itup
Lins
Novo
Ouri
Buri
Seve
Teju
Salt
Aran
Bert
Cerq
Echa
Água
Caça
Rose
Rege
Dirc
Bora
Ilha
Mira
Dour
Cris
Când
Barb
Jale
Jumi
Elis
Alvi
Plat
Port
Jeri
Borb
Taub
Taru
Marí
Guzo
Três
Itar
Brod
Pard
Igar
Báls
Para
Estr
Itaó
Jari
Mart
Mace
Meri
Cruz
Aruj
Momb
Cata
Brag
Bata
Sabi
Jard
Gabr
Balb
Avaí
Álva
Paca
Rifa
Baur
Sara
Espí
Lupé
Migu
Nara
Prom
Itan
Brej
Prac
Cesá


ProxyError: HTTPSConnectionPool(host='www.tjsp.jus.br', port=443): Max retries exceeded with url: /AutoComplete/ListarMunicipios (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 502 Proxy Error ( Forefront TMG denied the specified Uniform Resource Locator (URL).  )')))

In [ ]:
# Concatena todos os Resultados
df_tjsp = pd.concat(list_dfs, ignore_index=True)

# Ajusta a tabela
df_tjsp = df_tjsp.drop_duplicates()
df_tjsp = df_tjsp.sort_values(by='Descricao')
df_tjsp = df_tjsp.iloc[df_tjsp['Descricao'].str.normalize('NFKD').argsort()]
df_tjsp = df_tjsp.reset_index(drop=True)

# Resultados
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Salva
df_tjsp.to_csv(
    output_path_tab / 'Dim_Municipios.csv',
    index=False,
)
df_tjsp.to_excel(
    output_path_tab / 'Dim_Municipios.xlsx',
    sheet_name='Dim_Municipios',
    index=False,
)

<br>

## Lista de Foruns

Uma vez com a Lista de todos os múnicípios e seus respectivos códigos, foi possível obter as informações dos fóruns por meio de outro método POST.


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Dim_Municipios.csv')

# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Pesquisa pelo Município
df_tjsp[df_tjsp['Descricao'].str.startswith('Santos')]
df_tjsp[df_tjsp['Descricao'].str.startswith('Pereir')]
df_tjsp[df_tjsp['Descricao'].str.startswith('São João do Pau d')]
df_tjsp[df_tjsp['Descricao'].str.startswith('São Paulo')]

# Pesquisa pelo Código
# df_tjsp[df_tjsp['Codigo'] == 7053]

In [ ]:
def get_lista_unidades_tjsp(cod_municipio):
    """
    Pega a lista de unidades (Fóruns) de um determinado Município,
    a partir do Código do Município do TJSP

    :param cod_municipio: _description_
    :type cod_municipio: _type_
    :return: _description_
    :rtype: _type_
    """
    # Requests
    r = requests.post(
        'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
        json={'parmsEntrada': cod_municipio, 'codigoTipoBusca': 1},
    )
    # r.headers

    # BS4
    soup = BeautifulSoup(r.text, 'html.parser')
    # print(html.unescape(r.text))
    # soup.prettify()
    text_comarca = soup.find_all('h4')
    if text_comarca == []:
        text_comarca = 'É Município-Sede da Comarca'

    else:
        text_comarca = one(text_comarca)
        text_comarca = text_comarca.text
        text_comarca = text_comarca.split('jurisdicionado à comarca ')[-1]
        # print(text_comarca)
        # print(text_comcarca.split('jurisdicionado à comarca '))
    lista_unidades = [x.text for x in soup.find_all('span')]
    return lista_unidades, text_comarca

In [ ]:
# Apenas Exemplo da Função!
lista_unidades, comarca = get_lista_unidades_tjsp(7046)
lista_unidades, comarca = get_lista_unidades_tjsp(7053)
lista_unidades, comarca = get_lista_unidades_tjsp(7055)
print(lista_unidades, comarca)

<br>

Uma vez com a função que obter os dados das Unidades e, PRINCIPALMENTE, das comarcas!
Foi possível iterar pelos 645 municípios para pegar as informações!

> IMPORTANTE: FUNÇÃO LEVA UNS 4 MINUTOS PARA RODAR!


In [ ]:
df_tjsp['Comarca'] = None
df_tjsp['Unidades'] = None

for index, row in df_tjsp.iterrows():
    print(row['Descricao'])
    lista_unidades, comarca = get_lista_unidades_tjsp(row['Codigo'])
    df_tjsp.at[index, 'Comarca'] = comarca
    df_tjsp.at[index, 'Unidades'] = lista_unidades

In [ ]:
df_tjsp[df_tjsp['Comarca'] == 'É Município-Sede da Comarca']
# df_tjsp[df_tjsp['Comarca'] != 'É Município-Sede da Comarca']

In [ ]:
# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Salva
df_tjsp.to_csv(
    output_path_tab / 'Dim_Municipios.csv',
    index=False,
)
df_tjsp.to_excel(
    output_path_tab / 'Dim_Municipios.xlsx',
    sheet_name='Dim_Municipios',
    index=False,
)

<br>

## Análise dos Nomes dos Municípios

A ideia principal é agregar os códigos do IBGE na tabela. Para isso eu fiz um join... vi os erros... e corrigi!


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Dim_Municipios.csv')

# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Fiz isso para descobrir quais os nomes que não batem com a tabela "padrão"
df_mun_tjsp = df_tjsp[['Descricao']]
df_mun_tjsp.head()

<br>

Peguei a tabela padrão, que fiz!, no projeto do OpenGeo!


In [ ]:
df_geo_mun.info()
df_geo_mun.head()

<br>

Juntei


In [ ]:
df_merged = pd.merge(
    left=df_geo_mun,
    right=df_mun_tjsp,
    left_on='municipio_nome',
    right_on='Descricao',
    how='left',
)
df_merged[df_merged['Descricao'].isnull()]

In [ ]:
# Fiz apenas consultas, para ver como está escrito no TJSP
df_tjsp[df_tjsp['Descricao'].str.startswith('Estrela')]
df_tjsp[df_tjsp['Descricao'].str.startswith('Luís')]
df_tjsp[df_tjsp['Descricao'].str.startswith('Florí')]

<br>

Uma vez que sei os erros dos nomes e são apenas 3 registros...
Criei uma coluna duplicada para corrigir!


In [ ]:
df_tjsp['Nome Corrigido'] = df_tjsp['Descricao']

# Ajuste os Valores Errados
df_tjsp['Nome Corrigido'] = df_tjsp['Nome Corrigido'].replace(
    {
        'Estrela dOeste': "Estrela d'Oeste",
        'Luís Antônio': 'Luiz Antônio',
        'Florínia': 'Florínea',
    }
)

In [ ]:
# Checa se foi substituido na coluna "Nome Corrigido"
df_tjsp[df_tjsp['Nome Corrigido'].str.startswith('Florí')]

<br>

Por fim, checo novamente e vejo que não tem mais nada nulo! Excelente!


In [ ]:
df_merged = pd.merge(
    left=df_geo_mun,
    right=df_tjsp,
    left_on='municipio_nome',
    right_on='Nome Corrigido',
    how='left',
)

#
display(df_merged[df_merged['Descricao'].isnull()])


# Results
df_merged.info()
df_merged.head()

In [ ]:
df_tjsp[df_tjsp['Comarca'] == 'É Município-Sede da Comarca']

In [ ]:
df_merged = df_merged.drop(
    ['Nome Corrigido', 'Unidades'], axis=1, errors='ignore'
)

In [ ]:
df_merged = df_merged.rename(
    {
        'id_municipio': 'municipio_id',
        'Comarca': 'comarca',
        'Codigo': 'municipio_cod_tjsp',
        #'Codigo': 'cod_mun_tjsp',
        'Descricao': 'municipio_nome_tjsp',
    },
    axis=1,
)

# Results
df_merged.info()
df_merged.head()

In [ ]:
# Salva
df_merged.to_csv(
    output_path_tab / 'Dim_Municipios.csv',
    index=False,
)
df_merged.to_excel(
    output_path_tab / 'Dim_Municipios.xlsx',
    sheet_name='Dim_Municipios',
    index=False,
)

<br>

## Análises das Comarcas


Inicialmente conferi se o nome dos Municípios definido pelo TJSP correspondem ao nome das Comarcas que levantei!

Felizmente, pelo que analisei, a resposta é sim!


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Dim_Municipios.csv')

# Results
df_tjsp.info()
df_tjsp.head()

<br>

Crio uma tabela temporária, apenas para conseguir, posteriormente, trazer os códigos do IBGE para a tabela das comarcas


In [ ]:
# Comarca
df_comarca = df_tjsp[df_tjsp['comarca'] == 'É Município-Sede da Comarca']
df_comarca = df_comarca[
    ['municipio_id', 'municipio_nome', 'municipio_nome_tjsp']
]

df_comarca = df_comarca.rename(
    {
        'municipio_id': 'comarca_id',
        'municipio_nome': 'comarca_nome',
        'municipio_nome_tjsp': 'comarca_nome_tjsp',
    },
    axis=1,
)

# Results
df_comarca.info()
df_comarca.head()

In [ ]:
df_comarca = df_comarca.iloc[
    df_comarca['comarca_nome'].str.normalize('NFKD').argsort()
]
df_comarca = df_comarca.reset_index(drop=True)

In [ ]:
# Salva
df_comarca.to_csv(
    output_path_tab / 'Dim_Comarcas.csv',
    index=False,
)
df_comarca.to_excel(
    output_path_tab / 'Dim_Comarcas.xlsx',
    sheet_name='Comarcas',
    index=False,
)

<br>

### Municípios


In [ ]:
# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
df_tjsp_merged = pd.merge(
    left=df_tjsp,
    right=df_comarca,
    left_on='comarca',
    right_on='comarca_nome_tjsp',
    how='left',
)


df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# Cria Máscara
mask = df_tjsp_merged['comarca_id'].isnull()

# O que for nulo, preenche!
df_tjsp_merged.loc[mask, 'comarca_id'] = df_tjsp_merged['municipio_id']
# df_tjsp_merged.loc[mask, 'comarca_nome'] = df_tjsp_merged['municipio_nome']
# df_tjsp_merged.loc[mask, 'comarca_nome_tjsp'] = df_tjsp_merged['municipio_nome_tjsp']
df_tjsp_merged['comarca_id'] = df_tjsp_merged['comarca_id'].astype('int')

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# Cria Coluna Padrão
df_tjsp_merged['comarca_munsede'] = 0

# Cria Máscara
mask = df_tjsp_merged['comarca'] == 'É Município-Sede da Comarca'
df_tjsp_merged.loc[mask, 'comarca_munsede'] = 1

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# sss
df_tjsp_merged = df_tjsp_merged[
    [
        'municipio_id',
        'municipio_nome',
        'municipio_cod_tjsp',
        'municipio_nome_tjsp',
        'comarca_id',
        'comarca_munsede',
    ]
]

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# Sort com Acentos
# https://stackoverflow.com/questions/50217764/in-python-how-to-sort-a-dataframe-containing-accents
df_tjsp_merged = df_tjsp_merged.iloc[
    df_tjsp_merged['municipio_nome'].str.normalize('NFKD').argsort()
]
df_tjsp_merged = df_tjsp_merged.reset_index(drop=True)

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# Salva
df_tjsp_merged.to_csv(
    output_path_tab / 'Dim_Municipios.csv',
    index=False,
)
df_tjsp_merged.to_excel(
    output_path_tab / 'Dim_Municipios.xlsx',
    sheet_name='Dim_Municipios',
    index=False,
)